# 4_1_07 — Congruencia entre ingresos netos por representación legal y remuneración declarada
| Escenario                                                                                                  | Condición                                                          | Resultado        |
| ---------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------ | ---------------- |
| No aplica (no es el declarante quien realiza la representación)                                            | La casilla marcada en representación NO corresponde a “Declarante” | ⚪ **NO_APLICA**  |
| Tiene representación legal, pero no reporta monto mensual neto                                             | Monto mensual = 0, vacío o nulo                                    | ⚪ **SIN_DATO**   |
| Tiene representación legal con monto mensual neto > 0, pero en DSP **II.3** y **II.5** ambos son 0 o nulos | ingresos II.3 `<= 0` **y** ingresos II.5 `<= 0`                    | 🔴 **NO_CUMPLE** |
| Tiene representación legal con monto mensual neto > 0 **y** en DSP existe ingreso > 0 en II.3 o II.5       | II.3 `> 0` **o** II.5 `> 0`                                        | 🟢 **CUMPLE**    |
| Información insuficiente                                                                                   | Falta tipo de representación, monto o datos de DSP                 | ⚪ **SIN_DATO**   |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "4_1_07_CONGRUENCIA_REPRESENTACION_INGRESOS"

# --- Funciones auxiliares ---

def extraer_valor(doc, path):
    """Extrae valores tolerando dicts, listas y anidamientos."""
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None

        if actual in (None, "", " "):
            return None

        return actual
    except:
        return None


def normalizar_texto(valor):
    if valor is None:
        return None
    return str(valor).strip().upper()


def extraer_float(valor):
    try:
        return float(valor)
    except:
        return None


# --- Evaluación principal ---

def evaluar_metrica(doc):

    # ---------------------------
    #  Datos de representación legal (DI)
    # ---------------------------

    tipo_repr = normalizar_texto(
        extraer_valor(doc, "declaracion.intereses.representacion.tipo")
    )

    monto_repr = extraer_float(
        extraer_valor(doc, "declaracion.intereses.representacion.montoMensual")
    )

    # ---------------------------
    #  Datos de ingresos (DSP)
    # ---------------------------

    ingreso_ii3 = extraer_float(
        extraer_valor(doc,
            "declaracion.situacionPatrimonial.ingresos.serviciosProfesionales.servicios.remuneracion"
        )
    )

    ingreso_ii5 = extraer_float(
        extraer_valor(doc,
            "declaracion.situacionPatrimonial.ingresos.otrosIngresos.remuneracionTotal"
        )
    )

    # ---------------------------
    #  Lógica institucional
    # ---------------------------

    # 1. Si la representación NO corresponde al declarante → NO_APLICA
    if tipo_repr is None or "DECLARANTE" not in tipo_repr:
        return "NO_APLICA"

    # 2. Si el declarante tiene representación pero no indica monto → SIN_DATO
    if monto_repr is None or monto_repr <= 0:
        return "SIN_DATO"

    # 3. Tiene representación legal con monto > 0:
    #    validar ingresos II.3 o II.5
    ingresos_validos = False

    if ingreso_ii3 is not None and ingreso_ii3 > 0:
        ingresos_validos = True

    if ingreso_ii5 is not None and ingreso_ii5 > 0:
        ingresos_validos = True

    if ingresos_validos:
        return "CUMPLE"

    return "NO_CUMPLE"


# --- Procesamiento MongoDB ---

def procesar_metrica_4_1_07():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.intereses.representacion": 1,
            "declaracion.situacionPatrimonial.ingresos": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}\n")
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass
        print("\nConexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_1_07()
